In [ ]:
#!/usr/local/bin/python3
# Author: Tomas Hodan (hodantom@cmp.felk.cvut.cz)
# Center for Machine Perception, Czech Technical University in Prague

# A script to render 3D object models into the test images. The models are
# rendered at the ground truth 6D poses that are provided with the test images.
# The visualizations are saved into the folder specified by "output_dir".

from pytless import inout, misc
import os
import numpy as np

import matplotlib.pyplot as plt
import imageio

import camtools, open3d as o3d


In [ ]:
scene_ids = [1] # Choose which scene_ids to render. Eg. range(1, 21)
device = 'canon' # options: 'primesense' (720x540), 'kinect' (720x540), 'canon' (2560x1920)
model_type = 'cad' # options: 'cad', 'reconst'
im_step = 100 # Consider every im_step-th image

# Path to the T-LESS dataset.
# Which you can download using the t-less_download.py script. 
data_path = '../t-less_v2'

# Path to the folder in which the images produced by this script will be saved
output_dir = os.path.join(data_path, 'output_check_poses_test_imgs')

In [ ]:
# Paths to the elements of the T-LESS dataset
model_path_mask = os.path.join(data_path, 'models_' + model_type, 'obj_{:02d}.ply')
scene_info_path_mask = os.path.join(data_path, 'test_{}', '{:02d}', 'info.yml')
scene_gt_path_mask = os.path.join(data_path, 'test_{}', '{:02d}', 'gt.yml')
rgb_path_mask = os.path.join(data_path, 'test_{}', '{:02d}', 'rgb', '{:04d}.{}')
depth_path_mask = os.path.join(data_path, 'test_{}', '{:02d}', 'depth', '{:04d}.png')
rgb_ext = {'primesense': 'png', 'kinect': 'png', 'canon': 'jpg'}
obj_colors_path = os.path.join('data', 'obj_rgb.txt')
vis_rgb_path_mask = os.path.join(output_dir, '{:02d}_{}_{}_{:04d}_rgb.png')
vis_depth_path_mask = os.path.join(output_dir, '{:02d}_{}_{}_{:04d}_depth_diff.png')

In [ ]:
# reload pytless modules to reflect possible changes during development
import importlib
importlib.reload(inout)

In [ ]:
misc.ensure_dir(output_dir)
obj_colors = inout.load_colors(obj_colors_path)

In [ ]:
obj_colors

In [ ]:
import PIL
PIL.__version__
from PIL import ImageDraw, Image

img = Image.new('RGB', (640, 480), (73, 109, 137))
#img = Image.fromarray(np.zeros((480, 640)))
color = (255, 255, 255)
rect = [408, 239, 118, 93]
draw = ImageDraw.Draw(img)
# If 'TypeError: must be real number, not tuple' then the image is grayscale!
draw.rectangle((rect[0], rect[1], rect[0] + rect[2], rect[1] + rect[3]), outline=color, fill=color)

img

In [ ]:
scene_id = 1

# Load info about the test images (including camera parameters etc.)
scene_info_path = scene_info_path_mask.format(device, scene_id)
scene_info = inout.load_info(scene_info_path)

scene_gt_path = scene_gt_path_mask.format(device, scene_id)
scene_gt = inout.load_gt(scene_gt_path)

# Load models of objects present in the scene
scene_obj_ids = set()
for gt in scene_gt[0]:
    scene_obj_ids.add(gt['obj_id'])
models = {}
for scene_obj_id in scene_obj_ids:
    model_path = model_path_mask.format(scene_obj_id)
    models[scene_obj_id] = inout.load_ply(model_path)

In [ ]:
scene_obj_ids

In [ ]:
models[25].keys()

In [ ]:
models[25]["pts"].shape

In [ ]:
scene_info.keys()

In [ ]:
im_id, im_info = list(scene_info.items())[0]
im_id, im_info

In [ ]:
print('scene: ' + str(scene_id) + ', device: ' + device + ', im_id: ' + str(im_id))

# Get intrinsic camera parameters
K = im_info['cam_K']
print(f"cx, cy: {K[0,2]}, {K[1,2]}")

In [ ]:
rgb_path = rgb_path_mask.format(device, scene_id, im_id, rgb_ext[device])
rgb = imageio.v2.imread(rgb_path)

In [ ]:
plt.imshow(rgb)

In [ ]:
scene_gt.keys()

In [ ]:
scene_gt[9]

In [ ]:
def render_simple(model, im_size, K, R, t, z_in_view_space=True):
    # Create mesh
    mesh = o3d.geometry.TriangleMesh()
    mesh.vertices = o3d.utility.Vector3dVector(model['pts'])
    mesh.triangles = o3d.utility.Vector3iVector(model['faces'])
    mesh.compute_vertex_normals()

    # Set up renderer
    renderer = o3d.visualization.rendering.OffscreenRenderer(im_size[0], im_size[1])

    # Set up material
    material = o3d.visualization.rendering.MaterialRecord()
    material.shader = "defaultLit"

    renderer.scene.add_geometry("mesh", mesh, material)

    # Set camera intrinsics
    renderer.setup_camera(K, np.vstack([np.hstack([R, t]), np.array([0,0,0,1])]), im_size[0], im_size[1])

    # Render
    rgb = np.asarray(renderer.render_to_image()) # RGB image, H x W x 3, float, [0.0, 1.0]
    depth = np.asarray(renderer.render_to_depth_image(z_in_view_space=z_in_view_space)) # Depth image, H x W, float, in meters
    mask = depth < np.inf

    return rgb, depth, mask

In [ ]:
im_size = (rgb.shape[1], rgb.shape[0])
vis_rgb = np.zeros(rgb.shape, float)

rendered_depths = []
rendered_masks = []
rendered_obj_ids = []

for gt in scene_gt[im_id]:
    model = models[gt['obj_id']]
    R = gt['cam_R_m2c']
    t = gt['cam_t_m2c']
    surf_color = obj_colors[gt['obj_id'] - 1]
    print(f"surf_color: {surf_color}, obj_id: {gt['obj_id']}, model_type: {model}, K: {K}, R: {R}, t: {t}")

    # ren_rgb = renderer.render(model, im_size, K, R, t,
    #                             surf_color=surf_color, mode='rgb')
    ren_rgb, ren_depth, ren_mask = render_simple(model, im_size, K, R, t,)

    # check for empty array
    if ren_rgb.shape == (0,):
        print("EMPTY RENDERING!")
    else:
        print(ren_rgb.shape)

    rendered_depths.append(ren_depth)
    rendered_masks.append(ren_mask)
    rendered_obj_ids.append(gt['obj_id'])

    # Draw the bounding box of the object
    #print(gt['obj_bb'])
    #ren_rgb = misc.draw_rect(ren_rgb, gt['obj_bb'])

    #vis_rgb += 0.7 * ren_rgb.astype(float)

acc_depth = np.zeros(rgb.shape[:2], float) + np.inf
for ren_depth, ren_mask in zip(rendered_depths, rendered_masks):
    acc_depth[ren_mask] = np.minimum(acc_depth[ren_mask], ren_depth[ren_mask])
acc_depth[acc_depth == np.inf] = 0

object_masks = np.zeros(rgb.shape, int)
for idx, ren_mask in enumerate(rendered_masks):
    object_masks[(acc_depth <= rendered_depths[idx]) & (ren_mask == True)] = np.array(obj_colors[rendered_obj_ids[idx] - 1]) * 255
    print(f"Using object id {rendered_obj_ids[idx]} with color {obj_colors[rendered_obj_ids[idx] - 1]}")

plt.imshow(object_masks)#, cmap='gray', vmin=0, vmax=len(rendered_masks))

In [ ]:
%matplotlib inline
plt.imshow(rgb / 255 * 0.3 + 0.7 * object_masks / 255)

In [ ]:
rgb.shape

In [ ]:
K[:,2]

In [ ]:
# Visualize current model and camera pose in model coordinate system

T = np.eye(4)

# invert R, t to get camera to model transformation
T[:3, :3] = R # np.linalg.inv(R)
T[:3, 3] = (t).flatten() / 1000.0  # Convert mm to meters
camera_frustums = camtools.camera.create_camera_frustums([K], [T], size=1.)

model_mesh = o3d.geometry.TriangleMesh()
model_mesh.vertices = o3d.utility.Vector3dVector(models[gt['obj_id']]['pts']/1000)
model_mesh.triangles = o3d.utility.Vector3iVector(models[gt['obj_id']]['faces'])
model_mesh.compute_vertex_normals()

# draw world origin
axis_length = 0.1
world_origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=axis_length, origin=[0,0,0])
model_pcls = []#o3d.geometry.PointCloud(o3d.utility.Vector3dVector(model['pts']/1000+ np.array([0,0,idx+1]))) for idx, model in enumerate(models.values())]

o3d.visualization.draw_geometries([camera_frustums, world_origin, model_mesh, o3d.geometry.PointCloud(o3d.utility.Vector3dVector(model['pts']/1000))],)

In [ ]:
for scene_id in scene_ids:

    # Load info about the test images (including camera parameters etc.)
    scene_info_path = scene_info_path_mask.format(device, scene_id)
    scene_info = inout.load_info(scene_info_path)

    scene_gt_path = scene_gt_path_mask.format(device, scene_id)
    scene_gt = inout.load_gt(scene_gt_path)

    # Load models of objects present in the scene
    scene_obj_ids = set()
    for gt in scene_gt[0]:
        scene_obj_ids.add(gt['obj_id'])
    models = {}
    for scene_obj_id in scene_obj_ids:
        model_path = model_path_mask.format(scene_obj_id)
        models[scene_obj_id] = inout.load_ply(model_path)

    for im_id, im_info in scene_info.items():
        if im_id % im_step != 0:
            continue
        print('scene: ' + str(scene_id) + ', device: ' + device + ', im_id: ' + str(im_id))

        # Get intrinsic camera parameters
        K = im_info['cam_K']

        # Visualization #1
        #-----------------------------------------------------------------------
        # Load RGB image
        rgb_path = rgb_path_mask.format(device, scene_id, im_id, rgb_ext[device])
        rgb = imageio.v2.imread(rgb_path)

        im_size = (rgb.shape[1], rgb.shape[0])

        rendered_depths = []
        rendered_masks = []
        rendered_obj_ids = []

        #vis_rgb = np.zeros(rgb.shape, float)
        for gt in scene_gt[im_id]:
            model = models[gt['obj_id']]
            R = gt['cam_R_m2c']
            t = gt['cam_t_m2c']
            surf_color = obj_colors[gt['obj_id'] - 1]
            print(f"surf_color: {surf_color}, obj_id: {gt['obj_id']}, model_type: {model}, K: {K}, R: {R}, t: {t}")
            ren_rgb, ren_depth, ren_mask = render_simple(model, im_size, K, R, t,)

            # check for empty array
            if ren_rgb.shape == (0,):
                print("EMPTY RENDERING!")
            else:
                print(ren_rgb.shape)

            rendered_depths.append(ren_depth)
            rendered_masks.append(ren_mask)
            rendered_obj_ids.append(gt['obj_id'])

            # Draw the bounding box of the object
            #print(gt['obj_bb'])
            #ren_rgb = misc.draw_rect(ren_rgb, gt['obj_bb'])

            #vis_rgb += 0.7 * ren_rgb.astype(float)

        acc_depth = np.zeros(rgb.shape[:2], float) + np.inf
        for ren_depth, ren_mask in zip(rendered_depths, rendered_masks):
            acc_depth[ren_mask] = np.minimum(acc_depth[ren_mask], ren_depth[ren_mask])
        acc_depth[acc_depth == np.inf] = 0

        object_masks = np.zeros(rgb.shape, int)
        for idx, ren_mask in enumerate(rendered_masks):
            object_masks[(acc_depth >= rendered_depths[idx]) & (ren_mask == True)] = np.array(obj_colors[rendered_obj_ids[idx] - 1]) * 255
            print(f"Using object id {rendered_obj_ids[idx]} with color {obj_colors[rendered_obj_ids[idx] - 1]}")

        plt.imshow(object_masks)#, cmap='gray', vmin=0, vmax=len(rendered_masks))

        # Save the visualization
        vis_rgb = 0.6 * object_masks + 0.4 * rgb
        vis_rgb_path = vis_rgb_path_mask.format(scene_id, device, model_type, im_id)
        imageio.imwrite(vis_rgb_path, vis_rgb.astype(np.uint8))

        # Visualization #2
        #-----------------------------------------------------------------------
        if False and device != 'canon':
            # Load depth image
            depth_path = depth_path_mask.format(device, scene_id, im_id, rgb_ext[device])
            depth = imageio.v2.imread(depth_path)  # Unit: 0.1 mm
            depth = depth.astype(float) * 0.1  # Convert to mm

            # Render the objects at the ground truth poses
            im_size = (depth.shape[1], depth.shape[0])
            ren_depth = np.zeros(depth.shape, float)
            for gt in scene_gt[im_id]:
                model = models[gt['obj_id']]
                R = gt['cam_R_m2c']
                t = gt['cam_t_m2c']

                # Render the current object
                ren_depth_obj = renderer.render(model, im_size, K, R, t, mode='depth')

                # Add to the final depth map only the parts of the surface that
                # are closer than the surfaces rendered before
                visible_mask = np.logical_or(ren_depth == 0, ren_depth_obj < ren_depth)
                mask = np.logical_and(ren_depth_obj != 0, visible_mask)
                ren_depth[mask] = ren_depth_obj[mask].astype(float)

            # Calculate the depth difference at pixels where both depth maps
            # are valid
            valid_mask = (depth > 0) * (ren_depth > 0)
            depth_diff = valid_mask * (depth - ren_depth.astype(float))

            # Save the visualization
            vis_depth_path = vis_depth_path_mask.format(scene_id, device,
                                                        model_type, im_id)
            plt.matshow(depth_diff)
            plt.title('captured - rendered depth [mm]')
            plt.colorbar()
            plt.savefig(vis_depth_path)
            plt.close()
